In [33]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#these were from the spotify app. the code won't run after changing these to upload to github publicly
cid = 'ClientID'
secret = 'ClientSecret'

auth_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(auth_manager=auth_manager, requests_timeout=60)

In [34]:
#the limit is hard-capped at 50, I have ~60 playlists
playlists1 = sp.user_playlists('ouroborousmv',limit=50, offset=0)
playlists2 = sp.user_playlists('ouroborousmv',limit=50, offset=50)

In [35]:
playlist_ids = []
for i, item in enumerate(playlists1['items']):
    #print("%d %s" % (63-i, item['name']))
    playlist_ids.append(item['id'])

for i, item in enumerate(playlists2['items']):
    #print("%d %s" % (13-i, item['name']))
    playlist_ids.append(item['id'])

In [36]:
#so the playlists number increases in chronological order.
playlist_ids = playlist_ids[::-1]

In [67]:
def featList(f):
    res = [f['danceability'], f['energy'], f['key'], f['loudness'], f['mode'], f['speechiness'], f['acousticness'],
           f['instrumentalness'], f['liveness'], f['valence'], f['tempo'], f['duration_ms'], f['time_signature']]
    return res

In [96]:
col_names = {0:'track_id', 1:'name', 2:'artist', 3:'danceability', 4:'energy', 5:'key', 6:'loudness', 7:'mode',
             8:'speechiness', 9:'acousticness', 10:'instrumentalness', 11:'liveness', 12:'valence',
             13:'tempo', 14:'duration_ms', 15:'time_sig'}

In [100]:
#Picking out the bits of data that spotify sends with what I actually want,
#storing to a python list, to be converted later to pandas dataframe.
#Apparently, it's inefficient to append rows to a pandas dataframe since it remakes the entire thing.

data = []
for pl_id in playlist_ids:    
    new_playlist = pd.DataFrame(sp.playlist_items(pl_id))
    for i in range(len(new_playlist)):
        new_id = new_playlist['items'][i]['track']['id']
        if new_id == None: continue
        new_track_name = new_playlist['items'][i]['track']['name']
        new_artist_name = new_playlist['items'][i]['track']['artists'][0]['name']
        new_feat = featList(sp.audio_features(new_id)[0])
        new_row = [new_id, new_track_name, new_artist_name]
        for ele in new_feat: new_row.append(ele)
        data.append(new_row)

In [101]:
df = pd.DataFrame(data)

In [103]:
final_df = df.rename(columns=col_names)

In [118]:
#forgot to get the time the song was added to playlist
date_added = []
for pl_id in playlist_ids:    
    new_playlist = pd.DataFrame(sp.playlist_items(pl_id))
    for i in range(len(new_playlist)):
        new_id = new_playlist['items'][i]['track']['id'] #so the dates line up with the rest of the data
        if new_id == None: continue
        date_added.append(new_playlist['items'][i]['added_at'])

In [120]:
#added a new song to last playlist... removing it from date_added
date_added.pop()

#adding column to the rest of the dataframe
final_df['date_added'] = date_added

In [122]:
final_df.to_csv('spotify_data.csv')